In [1]:
import json
import pandas as pd
from web3 import Web3, HTTPProvider
from tqdm.notebook import tqdm
from IPython.core.display import display, HTML

from config import ERC721_ANALYSIS_DATASET_NAME, SNAPSHOT_BLOCKNUMBER, AZIMUTH_POINTS_TABLE_NAME, PARITY_URL, SNAPSHOT_TS
from src.utils_bigquery import get_df, create_table_from_df

DROP_TABLES = True
CREATE_TABLES = True

AZIMUTH_TOKEN_CONTRACT_ADDRESS = '0x6ac07b7c4601b5ce11de8dfe6335b871c7c4dd4d'
AZIMUTH_CONTRACT_ADDRESS = "0x223c067F8CF28ae173EE5CafEa60cA44C335fecB"

## Get List of Transferred Tokens

In [5]:
query_1 = f'''
SELECT
    DISTINCT value
FROM `bigquery-public-data.ethereum_blockchain.token_transfers`
WHERE token_address = '{AZIMUTH_TOKEN_CONTRACT_ADDRESS}'
  AND block_timestamp > '2019-01-03 00:00:00'
  AND block_timestamp < '{SNAPSHOT_TS}'
'''

azimuth_tokens_list = list(get_df(query=query_1).value)
print(f'Number of Transferred Azimuth Tokens {len(azimuth_tokens_list):>,}')

Number of Transferred Azimuth Tokens 5,144


## Get Point Owners and Types

In [6]:
with open("src/azimuth_abi.json") as f:
    AZIMUTH_CONTRACT_ABI = json.load(f)

w3 = Web3(HTTPProvider(PARITY_URL))
contract = w3.eth.contract(
    address=w3.toChecksumAddress(AZIMUTH_CONTRACT_ADDRESS),
    abi=AZIMUTH_CONTRACT_ABI)

In [7]:
POINT_DICT = {
    0: ['galaxy', 8],
    1: ['star', 8],
    2: ['planet', 16]
}
points_data = []
for point in tqdm(azimuth_tokens_list):
    owner = str(contract.functions.getOwner(int(point)).call({'defaultBlock': SNAPSHOT_BLOCKNUMBER}))
    point_size = contract.functions.getPointSize(int(point)).call()
    point_type = POINT_DICT[point_size][0]
    parent_point = int(point) // 2**POINT_DICT[point_size][1]
    points_data.append([point, point_size, point_type, owner, parent_point])
points_df = pd.DataFrame(points_data,
                         columns=['point', 'point_size', 'point_type', 'owner', 'parent_point'])
display(HTML(points_df.head().to_html(index=False, notebook=True, show_dimensions=False)))

  0%|          | 0/100 [00:00<?, ?it/s]

point,point_size,point_type,owner,parent_point
3208590752,2,planet,0x1C98188d632b64a15C6147bD4f8A503B457518c5,48959
3018326434,2,planet,0x451423bF9Cdeb98a8D310c2B9542c593A1b507C6,46056
1481114018,2,planet,0x797dC75B60CB63984c1221345D0C7B549cb7f978,22600
4182507938,2,planet,0x15c94F4fD1F73621e877aE7d80FC0b30886bD44F,63820
3406102946,2,planet,0xb684DA8411532c45448371460168c11A914b1245,51973


## Insert Point Data

In [8]:
create_table_from_df(source_df=points_df,
                     dataset_name=ERC721_ANALYSIS_DATASET_NAME,
                     table_name=AZIMUTH_POINTS_TABLE_NAME,
                     drop_existing_table=DROP_TABLES)

Table erc721_analysis:azimuth_points has been deleted.


True